## resnet19 finetuning Example

In [1]:
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import math
import numpy as np
import tensorflow as tf
import time
from matplotlib.image import imread

from datasets import dataset_utils

# Main slim library
slim = tf.contrib.slim

### Flower dataset Download and make tfrecord


In [2]:
import tensorflow as tf
from datasets import dataset_utils

url = "http://download.tensorflow.org/data/flowers.tar.gz"
flowers_data_dir = '/tmp/flowers'

if not tf.gfile.Exists(flowers_data_dir):
    tf.gfile.MakeDirs(flowers_data_dir)

dataset_utils.download_and_uncompress_tarball(url, flowers_data_dir)

Data has apparently already been downloaded and unpacked.


### Download pretrained resnset152 model

In [3]:
url = "http://download.tensorflow.org/models/resnet_v1_152_2016_08_28.tar.gz"
ckpt_file = '/tmp/checkpoints'

In [4]:
dataset_utils.download_and_uncompress_tarball(url, ckpt_file)

Data has apparently already been downloaded and unpacked.


###  make load_batch 

In [5]:
def load_batch(dataset, batch_size=32, height=224, width=224, is_training=False):
    """Loads a single batch of data.
    
    Args:
      dataset: The dataset to load.
      batch_size: The number of images in the batch.
      height: The size of each image after preprocessing.
      width: The size of each image after preprocessing.
      is_training: Whether or not we're currently training or evaluating.
    
    Returns:
      images: A Tensor of size [batch_size, height, width, 3], image samples that have been preprocessed.
      images_raw: A Tensor of size [batch_size, height, width, 3], image samples that can be used for visualization.
      labels: A Tensor of size [batch_size], whose values range between 0 and dataset.num_classes.
    """
    data_provider = slim.dataset_data_provider.DatasetDataProvider(
        dataset, common_queue_capacity=32,
        common_queue_min=8)
    image_raw, label = data_provider.get(['image', 'label'])
    
    # Preprocess image for usage by Inception.
    image = inception_preprocessing.preprocess_image(image_raw, height, width, is_training=is_training)
    
    # Preprocess the image for display purposes.
    image_raw = tf.expand_dims(image_raw, 0)
    image_raw = tf.image.resize_images(image_raw, [height, width])
    image_raw = tf.squeeze(image_raw)
    
    # Batch it up.
    images, images_raw, labels = tf.train.batch(
          [image, image_raw, label],
          batch_size=batch_size,
          num_threads=1,
          capacity=2*batch_size)
    
    return images, images_raw, labels

### Fine-tunning

In [6]:
# Note that this may take several minutes.
import os

from datasets import flowers
from nets import resnet_v1
from preprocessing import inception_preprocessing


slim = tf.contrib.slim

image_size = resnet_v1.resnet_v1.default_image_size

ckpt_dir = '/tmp/checkpoints'
ckpt_file = os.path.join(ckpt_dir, 'resnet_v1_152.ckpt')

exclude_scopes = ['resnet_v1_152/logits']
train_scopes = ['resnet_v1_152/logits']

train_dir = '/tmp/resnetv1_finetuned/'

    
with tf.Graph().as_default():
    tf.logging.set_verbosity(tf.logging.INFO)

    dataset = flowers.get_split('train', flowers_data_dir)
    images, _, labels = load_batch(dataset,  batch_size=256, height=image_size, width=image_size)
    one_hot_labels = slim.one_hot_encoding(labels, dataset.num_classes)

    # Create the model, use the default arg scope to configure the batch norm parameters.
    with slim.arg_scope(resnet_v1.resnet_arg_scope()):
        logits, _ = resnet_v1.resnet_v1_152(images, num_classes=dataset.num_classes, is_training=False)
        logits = tf. squeeze(logits, [1, 2])

    # Define the variables to train scopes
    train_vars = [var for var in  tf.trainable_variables() if var.op.name.startswith(tuple(train_scopes))]

    # Define the variables to initialize by excluding trainning scopes and train
    init_vars = slim.get_variables_to_restore(exclude=exclude_scopes)
    init_assign_op, init_feed_dict = slim.assign_from_checkpoint(ckpt_file, init_vars)
    
    # Initialized funciont by checkpoint files
    def init_fn(sess):
        sess.run(init_assign_op, init_feed_dict)
    
    # Specify the loss function:
    slim.losses.softmax_cross_entropy(logits, one_hot_labels)
    total_loss = slim.losses.get_total_loss()

    # Specify the optimizer and create the train op with only last logits variables to train
    optimizer = tf.train.AdamOptimizer(learning_rate=1e-4)
    train_op = slim.learning.create_train_op(total_loss, optimizer, variables_to_train=train_vars)

    # Run the training:
    final_loss = slim.learning.train(
       train_op,
       logdir=train_dir,
       init_fn=init_fn,
       number_of_steps=10)
    
    print('Finished training. Last batch loss %f' % final_loss)

Instructions for updating:
Please switch to tf.summary.FileWriter. The interface and behavior is the same; this is just a rename.
INFO:tensorflow:Starting Session.
INFO:tensorflow:Starting Queues.
INFO:tensorflow:global_step/sec: 0
INFO:tensorflow:global step 1: loss = 2.1570 (4.16 sec/step)
INFO:tensorflow:global step 2: loss = 2.1310 (2.80 sec/step)
INFO:tensorflow:global step 3: loss = 2.0886 (2.88 sec/step)
INFO:tensorflow:global step 4: loss = 2.1083 (2.89 sec/step)
INFO:tensorflow:global step 5: loss = 2.1149 (2.87 sec/step)
INFO:tensorflow:global step 6: loss = 2.0640 (2.89 sec/step)
INFO:tensorflow:global step 7: loss = 2.1142 (2.85 sec/step)
INFO:tensorflow:global step 8: loss = 2.0656 (2.88 sec/step)
INFO:tensorflow:global step 9: loss = 2.0916 (2.88 sec/step)
INFO:tensorflow:global step 10: loss = 2.0644 (2.90 sec/step)
INFO:tensorflow:Stopping Training.
INFO:tensorflow:Finished training! Saving model to disk.
Finished training. Last batch loss 2.064390
